### Import Gurobi

In [1]:
#import gurobipy
from gurobipy import *
import numpy as np

### Create Model, Decision Variables & Objective Function

In [2]:
### Define parameters/coefficients for model
ShipCost =   [[9, 15, 8, 10],
              [7, 15, 14, 12],
              [5, 5, 11, 12],
              [3, 6, 8, 11]]

Capacity = [125, 100, 150, 120]

Demand = [110, 115, 135, 130]

ProdCost = [50, 57, 55, 61]

# Range of plants and mines
plants = len(Capacity) #should be 3
mines = len(Demand) #should be 4

In [3]:
m=Model()
x = {}

#variable for every mine-plant combo
for i in range(mines):
    for j in range(plants):
        x[(i,j)] = m.addVar(vtype=GRB.CONTINUOUS, name='x%d,%d' % (i,j))
        
m.update()

m.setObjective(quicksum(quicksum(ShipCost[i][j]*x[(i,j)] for i in range(mines)) for j in range(plants)) +
               quicksum(quicksum(ProdCost[i]*x[(i,j)] for i in range(mines)) for j in range(plants)),GRB.MINIMIZE) 

Academic license - for non-commercial use only - expires 2021-03-14
Using license file /Users/jessietam/gurobi.lic


### Define Constraints

In [4]:
##Add constraints
for i in range(mines):
    m.addConstr(quicksum(x[(i,j)] for j in range(plants)) <= Capacity[i])
    
for j in range(plants):
    m.addConstr(quicksum(x[(i,j)] for i in range(mines)) >= Demand[j])

m.optimize()

Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (mac64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 8 rows, 16 columns and 32 nonzeros
Model fingerprint: 0x90428fee
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [6e+01, 7e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+02, 2e+02]
Presolve time: 0.04s
Presolved: 8 rows, 16 columns, 32 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   4.900000e+02   0.000000e+00      0s
      10    3.0770000e+04   0.000000e+00   0.000000e+00      0s

Solved in 10 iterations and 0.07 seconds
Optimal objective  3.077000000e+04


### Print Results

In [5]:
#Print results
for v in m.getVars():
    print('%s %g' % (v.varName, v.x))
    
print('Obj: %g' % m.objVal)

x0,0 0
x0,1 0
x0,2 125
x0,3 0
x1,0 0
x1,1 0
x1,2 0
x1,3 100
x2,0 5
x2,1 115
x2,2 0
x2,3 30
x3,0 105
x3,1 0
x3,2 10
x3,3 0
Obj: 30770


### Shadow Price & Reduce Cost

In [6]:
# Get shadow prices
shadow_prices = m.getAttr('Pi')
m.printAttr('Pi')
# 7 binding constraints
# 3 capacity are binding
# all demand are binding


  Constraint           Pi 
-------------------------
          R0          -11 
          R1           -2 
          R2           -4 
          R4           64 
          R5           64 
          R6           69 
          R7           71 


In [7]:
# Get reduce cost
m.printAttr('rc')


    Variable           rc 
-------------------------
        x0,0            6 
        x0,1           12 
        x1,0            2 
        x1,1           10 
        x1,2            4 
        x2,2            1 
        x3,1            3 
        x3,3            1 
